<a href="https://colab.research.google.com/github/samarth2015/srip_selection/blob/main/ML_UCI_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here we are going to use Random Forest and SVM to classify the various activities based on the features provided by the authors and generated by us using the TSFEL library.



## Importing Libraries and downloading the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!wget "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"

--2025-02-19 19:43:01--  https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘human+activity+recognition+using+smartphones.zip’

human+activity+reco     [               <=>  ]  58.18M  16.1MB/s    in 4.5s    

2025-02-19 19:43:06 (12.9 MB/s) - ‘human+activity+recognition+using+smartphones.zip’ saved [61005872]



In [ ]:
# unzip the HAR file
!unzip "/content/human+activity+recognition+using+smartphones.zip"

Archive:  /content/human+activity+recognition+using+smartphones.zip
 extracting: UCI HAR Dataset.names   
 extracting: UCI HAR Dataset.zip     


In [ ]:
!unzip "/content/UCI HAR Dataset.zip"

Archive:  /content/UCI HAR Dataset.zip
   creating: UCI HAR Dataset/
  inflating: UCI HAR Dataset/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/UCI HAR Dataset/
  inflating: __MACOSX/UCI HAR Dataset/._.DS_Store  
  inflating: UCI HAR Dataset/activity_labels.txt  
  inflating: __MACOSX/UCI HAR Dataset/._activity_labels.txt  
  inflating: UCI HAR Dataset/features.txt  
  inflating: __MACOSX/UCI HAR Dataset/._features.txt  
  inflating: UCI HAR Dataset/features_info.txt  
  inflating: __MACOSX/UCI HAR Dataset/._features_info.txt  
  inflating: UCI HAR Dataset/README.txt  
  inflating: __MACOSX/UCI HAR Dataset/._README.txt  
   creating: UCI HAR Dataset/test/
   creating: UCI HAR Dataset/test/Inertial Signals/
  inflating: UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt  
   creating: __MACOSX/UCI HAR Dataset/test/
   creating: __MACOSX/UCI HAR Dataset/test/Inertial Signals/
  inflating: __MACOSX/UCI HAR Dataset/test/Inertial Signals/._body_acc_x_test.txt  
  inflating

In [ ]:
!pip install tsfel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.8 MB/s eta 0:00:00


In [ ]:
import tsfel

## Loading the data

#### Loading the train data into separate variables

In [ ]:
acc_x_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt', dtype=np.float32, delimiter=None)

acc_y_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt', dtype=np.float32, delimiter=None)

acc_z_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt', dtype=np.float32, delimiter=None)

gyro_x_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt', dtype=np.float32, delimiter=None)

gyro_y_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt', dtype=np.float32, delimiter=None)

gyro_z_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt', dtype=np.float32, delimiter=None)

totalacc_x_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt', dtype=np.float32, delimiter=None)

totalacc_y_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt', dtype=np.float32, delimiter=None)

totalacc_z_train = np.genfromtxt('/content/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt', dtype=np.float32, delimiter=None)

In [ ]:
acc_x_train.shape, acc_y_train.shape, acc_z_train.shape, gyro_x_train.shape, gyro_y_train.shape, gyro_z_train.shape, totalacc_x_train.shape, totalacc_y_train.shape, totalacc_z_train.shape

((7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128),
 (7352, 128))

In [ ]:
inertial_signals = [acc_x_train, acc_y_train, acc_z_train, gyro_x_train, gyro_y_train, gyro_z_train, totalacc_x_train, totalacc_y_train, totalacc_z_train]

In [ ]:
labels = np.genfromtxt('/content/UCI HAR Dataset/train/y_train.txt', dtype=np.int32, delimiter=None)

labels.shape

(7352,)

#### Loading the test data into separate variables

In [ ]:
acc_x_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_acc_x_test.txt", dtype=np.float32, delimiter=None)

acc_y_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt", dtype=np.float32, delimiter=None)

acc_z_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt", dtype=np.float32, delimiter=None)

gyro_x_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt", dtype=np.float32, delimiter=None)

gyro_y_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt", dtype=np.float32, delimiter=None)

gyro_z_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt", dtype=np.float32, delimiter=None)

totalacc_x_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/total_acc_x_test.txt", dtype=np.float32, delimiter=None)

totalacc_y_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt", dtype=np.float32, delimiter=None)

totalacc_z_test = np.genfromtxt("/content/UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt", dtype=np.float32, delimiter=None)

In [ ]:
labels_test = np.genfromtxt("/content/UCI HAR Dataset/test/y_test.txt", dtype=np.int32, delimiter=None)
labels_test.shape

(2947,)

In [ ]:
X_train_original = np.loadtxt("/content/UCI HAR Dataset/train/X_train.txt")
X_test_original = np.loadtxt("/content/UCI HAR Dataset/test/X_test.txt")

X_train_original.shape, X_test_original.shape

((7352, 561), (2947, 561))

## Creating the custom features using TSFEL Library

#### Creating features using TSFEL
**Some of the TSFEL features that we can use are listed below**

- positive_turning
- negative_turning
- spectral_centroid
- spectral_distance
- spectral_slope
- spectral_rolloff
- spectral_rollon
- pk_pk_distance
- rms
- slope
- neighbourhood_peaks
- zero_cross
- wavelet_abs_mean
- wavelet_energy
- wavelet_entropy
- wavelet_std
- wavelet_var

In [ ]:
positive_turning = []
for activity in inertial_signals:
    temp = []
    for i in range(0, len(activity)):
        temp.append(tsfel.feature_extraction.features.positive_turning(activity[i]))
    temp = np.array(temp)
    positive_turning.append(temp)

positive_turning = np.array(positive_turning)

In [ ]:
positive_turning.shape, positive_turning[1].shape

((9, 7352), (7352,))

In [ ]:
#we cannot do that manually for each feature so writing a for loop
#but there are two types of functions some need sampling frequency and some don't
#below are the features which don't need sampling frequency
features_list = [
    "negative_turning",
    "pk_pk_distance",
    "rms",
    "slope",
    "neighbourhood_peaks",
    "zero_cross",
]

features_dict = {feature: [] for feature in features_list}

for activity in inertial_signals:
    temp_features = {feature: [] for feature in features_list}

    for i in range(len(activity)):
        for feature in features_list:
            func = getattr(tsfel.feature_extraction.features, feature)
            temp_features[feature].append(func(activity[i]))

    for feature in features_list:
        features_dict[feature].append(np.array(temp_features[feature]))

for feature in features_list:
    features_dict[feature] = np.array(features_dict[feature])


In [ ]:
features_dict['negative_turning'].shape

(9, 7352)

In [ ]:
#Here are the features which need sampling frequency
features_list2 = [
    "spectral_centroid",
    "spectral_distance",
    "spectral_slope",
    "spectral_roll_off",
    "spectral_roll_on",
    "wavelet_abs_mean",
    "wavelet_energy",
    "wavelet_entropy",
    "wavelet_std",
    "wavelet_var"
]

features_dict2 = {feature: [] for feature in features_list2}

for activity in inertial_signals:
    temp_features = {feature: [] for feature in features_list2}

    for i in range(len(activity)):
        for feature in features_list2:
            func = getattr(tsfel.feature_extraction.features, feature)
            temp_features[feature].append(func(activity[i], 8))

    for feature in features_list2:
        features_dict2[feature].append(np.array(temp_features[feature]))

for feature in features_list2:
    features_dict2[feature] = np.array(features_dict2[feature])


In [ ]:
features_dict2['spectral_centroid'].shape

(9, 7352)

#### Combining both the dicts and finally making X_train_custom

In [ ]:
features_dict['positive_turning'] = positive_turning

for feature in features_list2:
    features_dict[feature] = features_dict2[feature]


In [ ]:
features_dict.keys()

dict_keys(['negative_turning', 'pk_pk_distance', 'rms', 'slope', 'neighbourhood_peaks', 'zero_cross', 'positive_turning', 'spectral_centroid', 'spectral_distance', 'spectral_slope', 'spectral_roll_off', 'spectral_roll_on', 'wavelet_abs_mean', 'wavelet_energy', 'wavelet_entropy', 'wavelet_std', 'wavelet_var'])

In [ ]:
for key in features_dict.keys():
    features_dict[key] = np.transpose(features_dict[key])

In [ ]:
features_dict['negative_turning'].shape

(9, 7352)

In [ ]:
X_train_custom = np.concatenate([([value for value in feature]) for feature in features_dict.values()])

In [ ]:
X_train_custom.shape

(153, 7352)

In [ ]:
X_train_custom = np.transpose(X_train_custom)
X_train_custom.shape

(7352, 153)

## Making a Decision Tree Model